# Entrenamiento de los modelos

In [1]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np
import pandas as pd

In [1]:
dfPath = "dataframe/train_df.csv"
train_df = pd.read_csv(dfPath)

NameError: name 'pd' is not defined

In [3]:
train_df

,id,sentence_num,model,domain,POS_VERB,POS_NOUN,POS_ADJ,POS_ADV,POS_DET,POS_INTJ,...,RE,ASF,ASM,OM,RCI,DMC,OR,QAS,PA,PR
0,0,0,mistral,books,0.121212,0.212121,0.060606,0.090909,0.151515,0.0,...,0.0,0.0,0.030303,0.00,0.0,0.0,0.0,0.121212,0.0,0.000000
1,0,1,mistral,books,0.240000,0.160000,0.120000,0.040000,0.200000,0.0,...,0.0,0.0,0.000000,0.04,0.0,0.0,0.0,0.160000,0.0,0.040000
2,0,2,mistral,books,0.222222,0.111111,0.111111,0.000000,0.222222,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.111111,0.0,0.111111
3,0,3,mistral,books,0.285714,0.190476,0.047619,0.000000,0.142857,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.095238,0.0,0.000000
4,0,4,mistral,books,0.250000,0.062500,0.062500,0.000000,0.187500,0.0,...,0.0,0.0,0.062500,0.00,0.0,0.0,0.0,0.125000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10031,999,11,mistral,books,0.214286,0.142857,0.071429,0.000000,0.142857,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.071429,0.0,0.000000
10032,999,12,mistral,books,0.166667,0.166667,0.166667,0.000000,0.111111,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.055556,0.0,0.000000
10033,999,13,mistral,books,0.166667,0.166667,0.047619,0.047619,0.166667,0.0,...,0.0,0.0,0.023810,0.00,0.0,0.0,0.0,0.119048,0.0,0.000000
10034,999,14,mistral,books,0.250000,0.187500,0.062500,0.062500,0.187500,0.0,...,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.187500,0.0,0.000000


In [4]:
print("Información del dataset de entrenamiento:")
print(f"Forma del dataset: {train_df.shape}")
print(f"Columnas: {list(train_df.columns)}")
print(f"Modelos unicos: {train_df['model'].unique()}")
print(f"Dominios unicos: {train_df['domain'].unique()}")

Información del dataset de entrenamiento:
Forma del dataset: (10036, 200)
Columnas: ['id', 'sentence_num', 'model', 'domain', 'POS_VERB', 'POS_NOUN', 'POS_ADJ', 'POS_ADV', 'POS_DET', 'POS_INTJ', 'POS_CONJ', 'POS_PART', 'POS_NUM', 'POS_PREP', 'POS_PRO', 'L_REF', 'L_HASHTAG', 'L_MENTION', 'L_RT', 'L_LINKS', 'L_CONT_A', 'L_FUNC_A', 'L_CONT_T', 'L_FUNC_T', 'L_PLURAL_NOUNS', 'L_SINGULAR_NOUNS', 'L_PROPER_NAME', 'L_PERSONAL_NAME', 'L_NOUN_PHRASES', 'L_PUNCT', 'L_PUNCT_DOT', 'L_PUNCT_COM', 'L_PUNCT_SEMC', 'L_PUNCT_COL', 'L_PUNCT_DASH', 'L_POSSESSIVES', 'L_ADJ_POSITIVE', 'L_ADJ_COMPARATIVE', 'L_ADJ_SUPERLATIVE', 'L_ADV_POSITIVE', 'L_ADV_COMPARATIVE', 'L_ADV_SUPERLATIVE', 'PS_CONTRADICTION', 'PS_AGREEMENT', 'PS_EXAMPLES', 'PS_CONSEQUENCE', 'PS_CAUSE', 'PS_LOCATION', 'PS_TIME', 'PS_CONDITION', 'PS_MANNER', 'SY_QUESTION', 'SY_NARRATIVE', 'SY_NEGATIVE_QUESTIONS', 'SY_SPECIAL_QUESTIONS', 'SY_TAG_QUESTIONS', 'SY_GENERAL_QUESTIONS', 'SY_EXCLAMATION', 'SY_IMPERATIVE', 'SY_SUBORD_SENT', 'SY_SUBORD_SENT

## Preparación de datos

In [5]:
# 1. Crear etiqueta binaria: 1 = Humano (model == 'human'), 0 = IA (resto)
print("Distribución de clases:")
print((train_df['model'] == 'human').value_counts())
print(f"\nTotal de oraciones: {len(train_df)}")

Distribución de clases:
model
False    9769
True      267
Name: count, dtype: int64

Total de oraciones: 10036


In [6]:
# 2. Definir columnas de features (excluir metadatos y target)
# Excluir: id, sentence_num, model, domain
metadata_cols = ['id', 'sentence_num', 'model', 'domain']
feature_columns = [col for col in train_df.columns if col not in metadata_cols]

print(f"\nTotal de features: {len(feature_columns)}")
print(f"Primeros 10 features: {feature_columns[:10]}")


Total de features: 196
Primeros 10 features: ['POS_VERB', 'POS_NOUN', 'POS_ADJ', 'POS_ADV', 'POS_DET', 'POS_INTJ', 'POS_CONJ', 'POS_PART', 'POS_NUM', 'POS_PREP']


In [7]:
# 3. Obtener IDs únicos y crear mapping de ID a clase
unique_ids = train_df['id'].unique()
print(f"\nTotal de documentos únicos: {len(unique_ids)}")

# Mapping de ID a clase (binaria: humano vs IA)
id_to_class = train_df.groupby('id')['model'].first().to_dict()


Total de documentos únicos: 1000


In [8]:
# Verificar cuántos documentos hay por clase
print("Distribución de documentos por clase:")
class_distribution = pd.Series(id_to_class.values()).value_counts()
print(class_distribution)
print(f"\nClase 0 (IA): {class_distribution.get(0, 0)} documentos")
print(f"Clase 1 (Humano): {class_distribution.get(1, 0)} documentos")

Distribución de documentos por clase:
mistral-chat    127
mpt             120
mistral         118
gpt2            116
mpt-chat        106
llama-chat      105
gpt4             64
gpt3             59
cohere-chat      55
cohere           52
chatgpt          52
human            26
Name: count, dtype: int64

Clase 0 (IA): 127 documentos
Clase 1 (Humano): 120 documentos


C:\Users\HP\AppData\Local\Temp\ipykernel_9304\198586682.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"\nClase 0 (IA): {class_distribution.get(0, 0)} documentos")
C:\Users\HP\AppData\Local\Temp\ipykernel_9304\198586682.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Clase 1 (Humano): {class_distribution.get(1, 0)} documentos")


In [9]:
# 4. Split de IDs (no de oraciones) - IMPORTANTE para evitar data leakage
ids_list = list(id_to_class.keys())
labels_list = [id_to_class[id_] for id_ in ids_list]

# Verificar si podemos usar stratify
class_counts = pd.Series(labels_list).value_counts()
can_stratify = class_counts.min() >= 2

if can_stratify:
    print("✓ Usando stratified split (mantiene proporción de clases)")
    train_ids, test_ids = train_test_split(
        ids_list, 
        test_size=0.2, 
        random_state=42, 
        stratify=labels_list
    )
else:
    print(f"⚠️ No se puede usar stratify (clase mínima: {class_counts.min()} documentos)")
    print("Usando split aleatorio sin stratify")
    train_ids, test_ids = train_test_split(
        ids_list, 
        test_size=0.2, 
        random_state=42, 
        stratify=None  # Sin estratificación
    )

print(f"\nDocumentos en train: {len(train_ids)}")
print(f"Documentos en test: {len(test_ids)}")

✓ Usando stratified split (mantiene proporción de clases)

Documentos en train: 800
Documentos en test: 200


In [11]:
# 5. Filtrar oraciones según los IDs
train_sentences = train_df[train_df['id'].isin(train_ids)]
test_sentences = train_df[train_df['id'].isin(test_ids)]

print(f"\nOraciones en train: {len(train_sentences)}")
print(f"Oraciones en test: {len(test_sentences)}")

# Verificar distribución en cada conjunto
print(f"\nDistribución en train:")
# print(train_sentences['is_human'].value_counts())
print(train_sentences['model'].value_counts())
print(f"\nDistribución en test:")
# print(test_sentences['is_human'].value_counts())
print(test_sentences['model'].value_counts())


Oraciones en train: 7968
Oraciones en test: 2068

Distribución en train:
model
gpt2            1279
llama-chat      1004
mistral          916
mistral-chat     890
mpt              867
gpt4             638
chatgpt          548
mpt-chat         498
cohere           466
cohere-chat      332
gpt3             317
human            213
Name: count, dtype: int64

Distribución en test:
model
mistral         317
mpt             302
llama-chat      286
gpt2            257
mistral-chat    227
mpt-chat        127
gpt4            126
chatgpt         123
cohere-chat      96
gpt3             78
cohere           75
human            54
Name: count, dtype: int64


## Preparación de features

In [12]:
# 6. Preparar X e y
X_train = train_sentences[feature_columns].values
y_train = train_sentences['model'].apply(lambda x: 1 if x == 'human' else 0).values
X_test = test_sentences[feature_columns].values
y_test = test_sentences['model'].apply(lambda x: 1 if x == 'human' else 0).values

print(f"\n{'='*60}")
print(f"Shape de X_train: {X_train.shape}")
print(f"Shape de y_train: {y_train.shape}")
print(f"Shape de X_test: {X_test.shape}")
print(f"Shape de y_test: {y_test.shape}")
print(f"{'='*60}")


Shape de X_train: (7968, 196)
Shape de y_train: (7968,)
Shape de X_test: (2068, 196)
Shape de y_test: (2068,)


In [13]:
# Verificar si hay valores NaN en los datos
print("Verificando valores NaN en el dataset...")
print(f"\nNaNs en train_df: {train_df.isna().sum().sum()}")
print(f"NaNs en feature_columns:")
nan_features = train_df[feature_columns].isna().sum()
nan_features_with_nans = nan_features[nan_features > 0]
if len(nan_features_with_nans) > 0:
    print(nan_features_with_nans)
else:
    print("No hay NaNs en las features ✓")

print(f"\nNaNs en X_train: {np.isnan(X_train).sum()}")
print(f"NaNs en X_test: {np.isnan(X_test).sum()}")
print(f"NaNs en y_train: {np.isnan(y_train).sum()}")
print(f"NaNs en y_test: {np.isnan(y_test).sum()}")

Verificando valores NaN en el dataset...

NaNs en train_df: 34
NaNs en feature_columns:
ST_HERDAN_TTR    34
dtype: int64

NaNs en X_train: 24
NaNs en X_test: 10
NaNs en y_train: 0
NaNs en y_test: 0


In [14]:
# Solución: Imputar valores NaN antes de entrenar
from sklearn.impute import SimpleImputer

# Opción 1: Imputar con la media de cada feature
imputer = SimpleImputer(strategy='mean')

# Ajustar el imputer con los datos de entrenamiento
X_train_clean = imputer.fit_transform(X_train)
# Transformar los datos de test con el mismo imputer
X_test_clean = imputer.transform(X_test)

print("✓ Valores NaN imputados con la media de cada feature")
print(f"NaNs en X_train_clean: {np.isnan(X_train_clean).sum()}")
print(f"NaNs en X_test_clean: {np.isnan(X_test_clean).sum()}")

✓ Valores NaN imputados con la media de cada feature
NaNs en X_train_clean: 0
NaNs en X_test_clean: 0


## Entrenamiento del modelo

In [15]:
print("\nEntrenando SVM con split a nivel de documento...")
print("Kernel: RBF | C: 1.0 | Gamma: scale")
print("="*60)

svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svm_model.fit(X_train_clean, y_train)

print("✓ Modelo entrenado exitosamente")


Entrenando SVM con split a nivel de documento...
Kernel: RBF | C: 1.0 | Gamma: scale
✓ Modelo entrenado exitosamente


## Evaluación

In [16]:
y_pred = svm_model.predict(X_test_clean)

print(f"\n{'='*60}")
print("RESULTADOS EN TEST SET (Split por Documento)")
print(f"{'='*60}")
print(f"\nAccuracy: {accuracy_score(y_test, y_pred):.4f}")

print("\n--- Classification Report ---")
print(classification_report(y_test, y_pred, target_names=['IA', 'Humano'], digits=4))

print("\n--- Confusion Matrix ---")
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(f"\nInterpretación:")
print(f"  TN (IA correctamente clasificada): {cm[0,0]}")
print(f"  FP (IA clasificada como Humano): {cm[0,1]}")
print(f"  FN (Humano clasificado como IA): {cm[1,0]}")
print(f"  TP (Humano correctamente clasificado): {cm[1,1]}")


RESULTADOS EN TEST SET (Split por Documento)

Accuracy: 0.9739

--- Classification Report ---
              precision    recall  f1-score   support

          IA     0.9739    1.0000    0.9868      2014
      Humano     0.0000    0.0000    0.0000        54

    accuracy                         0.9739      2068
   macro avg     0.4869    0.5000    0.4934      2068
weighted avg     0.9485    0.9739    0.9610      2068


--- Confusion Matrix ---
[[2014    0]
 [  54    0]]

Interpretación:
  TN (IA correctamente clasificada): 2014
  FP (IA clasificada como Humano): 0
  FN (Humano clasificado como IA): 54
  TP (Humano correctamente clasificado): 0


c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\HP\Desktop\tesisI\tests\ml_algo\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le